# HW 7 Euler's Equations


**Author: Alex Choi**
*April 9, 2025$

## Function to return primitive variables

In [19]:
import numpy as np

def U2W(U, g):
    """
    Python equivalent of the MATLAB function:
        function V = LOBJ(u, p)
    """
    # u1
    rho = U[0]
    # u = ρu / ρ = u2 / u1
    u = U[1] / U[0]
    # pressure
    p = (g-1) * (U[2] - U[1]**2 / (2 * U[0]))

    return np.array([rho, u, p])


## Function to return Eigenvectors

In [20]:
def eigsEuler(U, g):
    """
    Compute the three eigenvalues of the 1D Euler equations
    using the conservative state vector U = [rho, rho*u, E]
    and the specific heat ratio gamma = g.

    In MATLAB terms, from class notes:
      lam1 = u - a
      lam2 = u
      lam3 = u + a
    
    Parameters
    ----------
    U : array-like of length 3
        U[0] = rho (density)
        U[1] = rho * u (momentum)
        U[2] = E (total energy)
    g : float
        gamma (specific heat ratio), e.g. 1.4

    Returns
    -------
    lambda : numpy.ndarray of length 3
        [u - a, u, u + a]
    """
    # Convert U to the primitive variables [rho, u, p] via your U2W function
    w = U2W(U, g)
    rho = w[0]
    u   = w[1]
    p   = w[2]
    
    # Speed of sound a = sqrt(g * p / rho)
    a = np.sqrt(g * p / rho)
    
    # Eigenvalues  -> λ1 = u - a, λ2 = u, λ3 = u + a
    eval1 = u - a
    eval2 = u
    eval3 = u + a
    
    return np.array([eval1, eval2, eval3])


## Functinot to return the flux vector

In [21]:
def fluxEuler(U, g):
    """
    Compute the flux of the 1D Euler equations based on the state U.

    Recall from class notes (in MATLAB):
      F = (rho * u, rho * u^2 + p, u * (E + p))

    Parameters
    ----------
    U : array-like of length 3
        [rho, rho*u, E]
    g : float
        gamma, the specific heat ratio (e.g., 1.4)

    Returns
    -------
    F : numpy.ndarray of length 3
        [F1, F2, F3] corresponding to 
         rho*u, rho*u^2 + p, and u*(E + p).
    """

    # Convert from conservative variables (U) to primitive variables w = [rho, u, p]
    w = U2W(U, g)
    rho = w[0]
    u = w[1]
    p = w[2]

    # The total energy E is the third component of U
    E = U[2]

    # Components of the flux
    F1 = rho * u
    F2 = rho * (u**2) + p
    F3 = u * (E + p)

    return np.array([F1, F2, F3])

## Test out values:

In [22]:
from scipy.io import loadmat
    
# Specific heat ratio (gamma)
g = 1.4

# Load the MAT-file containing U.
# The MATLAB file should have the variable U (an N x 3 array).
mat_data = loadmat('U.mat')
U = mat_data['U']  # U is assumed to be an N x 3 matrix

n = U.shape[0]

# Initialize lists to store the results for each U vector
W_list = []       # for primitive variables
lambda_list = []  # for eigenvalues
F_list = []       # for fluxes

# Loop over each state (each row of U)
for i in range(n):
    state = U[i, :]          # a 3-element vector: [rho, rho*u, E]
    W_i = U2W(state, g)
    lambda_i = eigsEuler(state, g)
    F_i = fluxEuler(state, g)
    W_list.append(W_i)
    lambda_list.append(lambda_i)
    F_list.append(F_i)

# Convert lists to NumPy arrays for easier formatting if needed
W = np.array(W_list)
lambdas = np.array(lambda_list)
F = np.array(F_list)

# Print the results similar to MATLAB's fprintf formatting.
print("\n\n--- W ----")
for i in range(n):
    # Each value is printed in the %12.3e format
    print(f"{W[i,0]:12.3e} {W[i,1]:12.3e} {W[i,2]:12.3e}")

print("\n\n--- lambda ----")
for i in range(n):
    print(f"{lambdas[i,0]:12.3e} {lambdas[i,1]:12.3e} {lambdas[i,2]:12.3e}")

print("\n\n--- F ----")
for i in range(n):
    print(f"{F[i,0]:12.3e} {F[i,1]:12.3e} {F[i,2]:12.3e}")



--- W ----
   1.000e+00    7.500e-01    1.000e+00
   1.000e+00   -2.000e+00    4.000e-01
   1.250e-01    0.000e+00    1.000e-01
   5.992e+00   -6.196e+00    4.609e+01
   1.000e+00   -1.960e+01    1.000e-02


--- lambda ----
  -4.332e-01    7.500e-01    1.933e+00
  -2.748e+00   -2.000e+00   -1.252e+00
  -1.058e+00    0.000e+00    1.058e+00
  -9.478e+00   -6.196e+00   -2.915e+00
  -1.972e+01   -1.960e+01   -1.948e+01


--- F ----
   7.500e-01    1.562e+00    2.836e+00
  -2.000e+00    4.400e+00   -6.800e+00
   0.000e+00    1.000e-01    0.000e+00
  -3.713e+01    2.762e+02   -1.712e+03
  -1.960e+01    3.841e+02   -3.764e+03
